In [72]:
import numpy as np
import os
import pandas as pd
import xarray as xr

RESULTS_NAMES = ['train_acc1', 'val_acc1', 'train_loss', 'val_loss', 'learning_rate']


class FullExperiment:

    def __init__(self, folder_path=None, experiment_list=None):

        # TODO error if both inputs are None or not None
        # if folder_path is None and experiment_list is None:

        if folder_path is not None:
            self.list = load_experiments(folder_path)
        if experiment_list is not None:
            self.list = experiment_list

        self.len = len(self.list)

    def filter_parameters(self, parameters_dict):
        new_list = []
        for experiment in self.list:
            include = True
            for parameter, value in parameters_dict.items():
                if (parameter not in experiment.parameters_dict.keys() or 
                        experiment.parameters_dict[parameter] != value):
                    include = False
                    continue 
            if include:
                new_list.append(experiment)
        return FullExperiment(experiment_list=new_list)

    def sort_by_parameter(self, parameter, reverse):

        # TODO parameter options

        def sort_fun(elem):
            return elem.parameters_dict[parameter]

        self.list.sort(key=sort_fun, reverse=reverse)

    def sort_by_results(self, result, metric, reverse):

        # TODO result and metric options

        def sort_fun(elem):
            return elem.get_experiment_summary()[result][metric]

        self.list.sort(key=sort_fun, reverse=reverse)

    def make_latex_table(self, parameter_list, result_list, result_type_list, name, precision,):

        latex_table = '\\multirow'
        latex_table += '{' + str(self.len) + '}{*}'
        latex_table += '{' + name + '}\n'
        
        for experiment in self.list:
            for parameter in parameter_list:
                latex_table += '& ' + str(experiment.parameters_dict[parameter])

            experiment_summary = experiment.experiment_results()
            for result, result_type in zip(result_list, result_type_list):
                result_value = experiment_summary[result][result_type]
                result_str = str(result_value.round(precision));
                if len(result_str.split('.')[1]) != precision:
                    result_str += '0'
                
                result_str += ' ($\\pm$ '
                if 'min' in result_type:
                    sd_type = 'std_min_mean'
                else:
                    sd_type = 'std_max_mean'
                    
                sd_value = experiment_summary[result][sd_type]
                sd_str = str(sd_value.round(precision));
                if len(sd_str.split('.')[1]) != precision:
                    sd_str += '0'

                result_str += sd_str + ')'
                
                latex_table += ' & ' + result_str

            latex_table += ' \\\\ \n'
            
        latex_table += '\\midrule \n'

        return latex_table
    
    def show_results(self, parameters, result_names, result_types):
        for i_experiment, experiment in enumerate(self.list):
            print(10*'-', 'EXPERIMENT {}'.format(i_experiment), 10*'-')
            print(3*'*','PARAMETERS', 3*'*')
            if parameters == 'all':
                
                for key, value in experiment.parameters_dict.items():
                    print(key,': ', value)
            elif parameters == 'training':
                print('training method: ', experiment.parameters_dict['training_method'])
                for key, value in experiment.training_method_parameters.items():
                    print(key, ': ', value) 
            elif parameters == 'learning':
                print('learning method: ', experiment.parameters_dict['learning_method'])
                for key, value in experiment.learning_method_parameters.items():
                    print(key, ': ', value)
            elif parameters == 'T&L':
                print('training method: ', experiment.parameters_dict['training_method'])
                for key, value in experiment.training_method_parameters.items():
                    print(key, ': ', value) 
                print('learning method: ', experiment.parameters_dict['learning_method'])
                for key, value in experiment.learning_method_parameters.items():
                    print(key, ': ', value)
            else: # parameters is a list
                for parameter in parameters:
                    print(parameter, ': ', experiment.parameters_dict[parameter])
    
                
            print(3*'*', 'RESULTS', 3*'*')
            experiment_results = experiment.experiment_results()
            for result_name in result_names:
                for result_type in result_types:
                    print(result_name, experiment_results[result_name][result_type])
            

class SingleExperiment:

    def __init__(self, file_path):
        self.file_name = file_path.split('/')[-1]
        self.parameters_dict = get_parameters(file_path)
        self.results_df = get_results_pandas(file_path)
        self.results_ds = get_results_xarray(file_path)

        self.parameters_dict['epochs'] = int(self.parameters_dict['epochs'])
        self.parameters_dict['executions'] = int(self.parameters_dict['executions'])

        self.parameters_dict['bach_size'] = int(self.parameters_dict['batch_size'])

        self.parameters_dict['test_set_split'] = float(self.parameters_dict['test_set_split'])
        self.parameters_dict['validation_set_split'] = float(self.parameters_dict['validation_set_split'])
        self.parameters_dict['reduce_train_set'] = float(self.parameters_dict['reduce_train_set'])

        self.parameters_dict['base_seed'] = int(self.parameters_dict['base_seed'])
        
        training_method = self.parameters_dict['training_method']
        learning_method = self.parameters_dict['learning_method']

        self.training_method_parameters = {key: value for key, value
                                           in self.parameters_dict.items()
                                           if key.split('_')[0] == training_method}

        self.learning_method_parameters = {key: value for key, value
                                           in self.parameters_dict.items()
                                           if key.split('_')[0] == learning_method}
        
        for key, value in self.parameters_dict.items():
            if value == 'False':
                self.parameters_dict[key] = False
            elif value == 'True':
                self.parameters_dict[key] = True

    def experiment_results(self, epoch_limit=None):

        if epoch_limit is not None:
            self.results_ds.drop(range(self.parameters_dict['epochs'], epoch_limit, -1), dim='epoch')

        summary_dict = {}
        for result in RESULTS_NAMES:
            result_da = self.results_ds[result]

            mean_da = result_da.mean(dim='execution')
            std_da = result_da.std(dim='execution')
            max_da = result_da.max(dim='execution')
            min_da = result_da.min(dim='execution')

            max_mean = mean_da.max(dim='epoch').values
            max_mean_epoch = mean_da.argmax(dim='epoch').values + 1
            min_mean = mean_da.min(dim='epoch').values
            min_mean_epoch = mean_da.argmin(dim='epoch').values + 1

            summary = {'mean': mean_da.values, 'std': std_da.values,
                       'max': max_da.values, 'min': min_da.values,
                       'max_mean': max_mean, 'max_mean_epoch': max_mean_epoch,
                       'min_mean': min_mean, 'min_mean_epoch': min_mean_epoch,
                       'std_max_mean': std_da.sel(epoch=max_mean_epoch).values,
                       'std_min_mean': std_da.sel(epoch=min_mean_epoch).values}

            summary_dict[result] = summary

        return summary_dict


def get_parameters(file_path):
    with open(file_path, 'r') as f:
        parameters_dict = {}
        lines = f.readlines()
        for line in lines:
            line = line[:-1]

            # if we get to a blank line it means that the parameters have ended
            if len(line) == 0:
                break

            colon_position = line.find(':')

            if colon_position != -1:
                parameter = line[:colon_position]
                value = line[colon_position + 1:]
                try:
                    value = int(value)
                except (ValueError, TypeError) as error:
                    try:
                        value = float(value)
                    except (ValueError, TypeError)as error:
                        pass

                parameters_dict[parameter] = value

        return parameters_dict


def get_values(file_path, value_name, parameters_dict):
    n_epochs = int(parameters_dict['epochs'])
    n_executions = int(parameters_dict['executions'])
    result = np.empty((n_executions, n_epochs))
    i_execution = 0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        correct_section = False
        for line in lines:
            line = line[:-1]

            if line == value_name:
                correct_section = True
                continue

            if correct_section:
                line_list = line.split(',')
                if len(line) == 0:
                    break
                result[i_execution, :] = np.array(line_list, dtype=float)
                i_execution += 1
        return result


def get_results_xarray(file_path):
    parameters_dict = get_parameters(file_path)

    results_dict = {name: get_values(file_path, name, parameters_dict) for name in RESULTS_NAMES}
    n_executions = int(parameters_dict['executions'])
    n_epochs = int(parameters_dict['epochs'])

    results_ds = xr.Dataset()
    for key, value in results_dict.items():
        results_ds[key] = (('execution', 'epoch'), value)

    results_ds.coords['epoch'] = range(1, n_epochs + 1)
    results_ds.coords['execution'] = range(1, n_executions + 1)

    # results_df = results_ds.to_dataframe()
    # results_df = results_df.swaplevel('epoch', 'execution', axis=0)
    # results_df = results_df.sort_index(level=0)

    return results_ds


def get_results_pandas(file_path):
    parameters_dict = get_parameters(file_path)

    results_dict = {name: get_values(file_path, name, parameters_dict) for name in RESULTS_NAMES}
    n_results = len(RESULTS_NAMES)
    n_executions = int(parameters_dict['executions'])
    n_epochs = int(parameters_dict['epochs'])

    results_array = np.empty((n_results, n_executions, n_epochs))  # results, execution, epoch
    for ii, result in enumerate(results_dict.values()):
        results_array[ii] = result

    results_array = np.swapaxes(results_array, 2, 0)  # epochs, execution, results
    results_array = np.swapaxes(results_array, 0, 1)  # execution, epoch, result
    results_array = np.reshape(results_array, (n_executions * n_epochs, n_results), order='C')

    execution = [ii for ii in range(1, n_executions + 1)]
    epochs = [ii for ii in range(1, n_epochs + 1)]
    index = pd.MultiIndex.from_product([execution, epochs],
                                        names=['execution', 'epoch'])

    results_df = pd.DataFrame(results_array,
                              index=index,
                              columns=RESULTS_NAMES)

    return results_df


def load_experiments(folder_path):
    files_name = sorted(os.listdir(folder_path))
    # only load .txt files
    files_name = list(filter(lambda x: x[-3:] == 'txt', files_name))
    files_path = [folder_path + file_name for file_name in files_name]
    experiment_list = []
    for file_path in files_path:
        experiment = SingleExperiment(file_path)
        experiment_list.append(experiment)

    return experiment_list


if __name__ == '__main__':

    folder_path_t = '/home/pedro/PycharmProjects/Masters/new_project/results/first_experiment/data/'
    file_name_t = 'sgd_tas-2019-01-12 18:16:22.822506.txt'
    file_path_t = folder_path_t + file_name_t
    full_experiment = FullExperiment(folder_path=folder_path_t)
    sgd_experiments = full_experiment.filter_parameters({'training_method': 'sgd'})


    # latex_table = sgd_experiments.make_latex_table('initial_learning_rate', ['val_acc1', 'val_loss'], 'test')
    # print(latex_table)


    #
    # for experiment in full_experiment.list:
    #     print(experiment.get_experiment_summary()['val_acc1']['max_mean'])
    #
    # full_experiment.sort_by_results('val_acc1', 'max_mean', reverse=False)
    # print('-------------------------')
    # for experiment in full_experiment.list:
    #     print(experiment.get_experiment_summary()['val_acc1']['max_mean'])
    # #
    # # print(full_experiment.list[0].get_experiment_summary())




In [73]:
folder_path_t = '/home/pedro/PycharmProjects/Masters/new_project/results/first_experiment/data/'
file_name_t = 'sgd_tas-2019-01-12 18:16:22.822506.txt'
file_path_t = folder_path_t + file_name_t
full_experiment = FullExperiment(folder_path=folder_path_t)



In [34]:
# SGD experiments

sgd_experiments = full_experiment.filter_parameters({'training_method': 'sgd',
                                                    'learning_method':'constant'})

# see all sgd experiments

print(sgd_experiments.len)


# we have 25 experiments in total, 7 for sgd without moment 7 for sgd with moment 
# 7 for sgd with nesterov moment and more 5 (4 new and 1 already presented among the 
# 7 from the sgd with nesterov moment) testing different nesterov moments with the best learning rate.





25


In [4]:
# Tests varying the learning rate and fixed momentum 

sgd_no_momentum = sgd_experiments.filter_parameters({'sgd_momentum':False})

sgd_momentum = sgd_experiments.filter_parameters({'sgd_momentum':0.9,
                                                  'sgd_nesterov':False})

sgd_nesterov = sgd_experiments.filter_parameters({'sgd_momentum':0.9,
                                                  'sgd_nesterov':True})

experiment_list = [sgd_no_momentum, sgd_momentum, sgd_nesterov]
names_list = ['SGD', 'SGD Momentum', 'SGD Nesterov']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter='sgd_lr', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=2))


\multirow{7}{*}{SGD}
& 2 & 48.94 ($\pm$ 38.92) & 1.38 ($\pm$ 0.92) \\ 
& 1 & 89.07 ($\pm$ 0.04) & 0.45 ($\pm$ 0.00) \\ 
& 0.5 & 89.64 ($\pm$ 0.32) & 0.45 ($\pm$ 0.01) \\ 
& 0.25 & 90.48 ($\pm$ 0.23) & 0.39 ($\pm$ 0.01) \\ 
& 0.05 & 90.00 ($\pm$ 0.12) & 0.45 ($\pm$ 0.01) \\ 
& 0.01 & 86.41 ($\pm$ 0.23) & 0.53 ($\pm$ 0.03) \\ 
& 0.001 & 79.12 ($\pm$ 0.21) & 0.62 ($\pm$ 0.00) \\ 
\midrule 

\multirow{7}{*}{SGD Momentum}
& 2 & 10.12 ($\pm$ 0.06) & 2.57 ($\pm$ 0.07) \\ 
& 1 & 10.02 ($\pm$ 0.03) & 2.40 ($\pm$ 0.08) \\ 
& 0.5 & 28.96 ($\pm$ 19.08) & 1.84 ($\pm$ 0.47) \\ 
& 0.25 & 48.54 ($\pm$ 38.54) & 1.39 ($\pm$ 0.92) \\ 
& 0.05 & 91.08 ($\pm$ 0.44) & 0.36 ($\pm$ 0.01) \\ 
& 0.01 & 90.58 ($\pm$ 0.23) & 0.40 ($\pm$ 0.02) \\ 
& 0.001 & 86.54 ($\pm$ 0.34) & 0.52 ($\pm$ 0.02) \\ 
\midrule 

\multirow{7}{*}{SGD Nesterov}
& 2 & 10.04 ($\pm$ 0.00) & 2.56 ($\pm$ 0.21) \\ 
& 1 & 10.06 ($\pm$ 0.03) & 2.41 ($\pm$ 0.05) \\ 
& 0.5 & 10.02 ($\pm$ 0.04) & 2.31 ($\pm$ 0.00) \\ 
& 0.25 & 86.06 ($\pm$ 0.09) &

In [5]:
# Tests varying the momentum and fixing the learning rate.

sgd_test_momentum = sgd_experiments.filter_parameters({'sgd_lr':0.05, 'sgd_nesterov':True})
sgd_test_momentum.sort_by_parameter('sgd_momentum', reverse=False)

print(sgd_test_momentum.make_latex_table(parameter='sgd_momentum', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name='SGD Nesterov',
                                      precision=2))

\multirow{5}{*}{SGD Nesterov}
& 0.1 & 89.92 ($\pm$ 0.43) & 0.45 ($\pm$ 0.01) \\ 
& 0.3 & 90.31 ($\pm$ 0.13) & 0.42 ($\pm$ 0.00) \\ 
& 0.5 & 90.50 ($\pm$ 0.23) & 0.41 ($\pm$ 0.01) \\ 
& 0.7 & 90.84 ($\pm$ 0.13) & 0.39 ($\pm$ 0.01) \\ 
& 0.9 & 90.85 ($\pm$ 0.29) & 0.37 ($\pm$ 0.00) \\ 
\midrule 



In [33]:
# adam experiments

adam_experiments = full_experiment.filter_parameters({'training_method': 'adam',
                                                    'learning_method':'constant'})

# see all sgd experiments

print(adam_experiments.len)

# for experiment in adam_experiments.list:
#     print(experiment.parameters_dict)
#     print('\n')
    
    
# 12 tests 6 with amsgrad and 6 without.

12


In [30]:
adam_amsgrad = adam_experiments.filter_parameters({'adam_amsgrad':True})

adam_no_amsgrad = adam_experiments.filter_parameters({'adam_amsgrad':False})

experiment_list = [adam_no_amsgrad, adam_amsgrad]
names_list = ['ADAM', 'ADAM amsgrad']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter='adam_lr', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=3))

\multirow{6}{*}{ADAM}
& 0.005 & 90.365 ($\pm$ 0.255) & 0.408 ($\pm$ 0.007) \\ 
& 0.001 & 91.080 ($\pm$ 0.10) & 0.382 ($\pm$ 0.004) \\ 
& 0.0005 & 91.130 ($\pm$ 0.340) & 0.380 ($\pm$ 0.020) \\ 
& 0.0003 & 90.995 ($\pm$ 0.045) & 0.388 ($\pm$ 0.018) \\ 
& 0.0001 & 88.775 ($\pm$ 0.125) & 0.455 ($\pm$ 0.002) \\ 
& 5e-05 & 86.190 ($\pm$ 0.190) & 0.529 ($\pm$ 0.009) \\ 
\midrule 

\multirow{6}{*}{ADAM amsgrad}
& 0.005 & 90.610 ($\pm$ 0.230) & 0.403 ($\pm$ 0.014) \\ 
& 0.001 & 91.030 ($\pm$ 0.270) & 0.380 ($\pm$ 0.010) \\ 
& 0.0005 & 91.20 ($\pm$ 0.290) & 0.379 ($\pm$ 0.009) \\ 
& 0.0003 & 90.895 ($\pm$ 0.105) & 0.385 ($\pm$ 0.011) \\ 
& 0.0001 & 87.970 ($\pm$ 0.050) & 0.466 ($\pm$ 0.002) \\ 
& 5e-05 & 85.570 ($\pm$ 0.140) & 0.548 ($\pm$ 0.009) \\ 
\midrule 



In [31]:
# rmsprop experiments

rmsprop_experiments = full_experiment.filter_parameters({'training_method': 'rmsprop',
                                                    'learning_method':'constant'})

# see all sgd experiments

print(rmsprop_experiments.len)
    
# 12 tests 6 with centered rmsprop and 6 without.

12


In [27]:
rmsprop_centered = rmsprop_experiments.filter_parameters({'rmsprop_centered':True})

rsmprop_not_centered = rmsprop_experiments.filter_parameters({'rmsprop_centered':False})

experiment_list = [rsmprop_not_centered, rmsprop_centered]
names_list = ['RMSPROP', 'RMSPROP centered']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter='rmsprop_lr', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=3))

\multirow{6}{*}{RMSPROP}
& 0.01 & 89.850 ($\pm$ 0.010) & 0.421 ($\pm$ 0.001) \\ 
& 0.005 & 90.020 ($\pm$ 0.170) & 0.430 ($\pm$ 0.018) \\ 
& 0.001 & 90.805 ($\pm$ 0.155) & 0.410 ($\pm$ 0.003) \\ 
& 0.0005 & 90.950 ($\pm$ 0.080) & 0.386 ($\pm$ 0.011) \\ 
& 0.0003 & 90.610 ($\pm$ 0.250) & 0.402 ($\pm$ 0.005) \\ 
& 0.0001 & 89.210 ($\pm$ 0.010) & 0.458 ($\pm$ 0.014) \\ 
\midrule 

\multirow{6}{*}{RMSPROP centered}
& 0.01 & 90.305 ($\pm$ 0.215) & 0.434 ($\pm$ 0.003) \\ 
& 0.005 & 90.370 ($\pm$ 0.140) & 0.433 ($\pm$ 0.015) \\ 
& 0.001 & 90.655 ($\pm$ 0.425) & 0.412 ($\pm$ 0.009) \\ 
& 0.0005 & 90.720 ($\pm$ 0.030) & 0.386 ($\pm$ 0.013) \\ 
& 0.0003 & 90.815 ($\pm$ 0.215) & 0.408 ($\pm$ 0.018) \\ 
& 0.0001 & 88.955 ($\pm$ 0.045) & 0.465 ($\pm$ 0.003) \\ 
\midrule 



In [38]:
# adagrad experiments

adagrad_experiments = full_experiment.filter_parameters({'training_method': 'adagrad',
                                                    'learning_method':'constant'})

# see all sgd experiments

print(adagrad_experiments.len)

# for experiment in adagrad_experiments.list:
#     print(experiment.parameters_dict)
#     print('\n')
    
# 5 tests

5


In [40]:
experiment_list = [adagrad_experiments]
names_list = ['ADAGRAD']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter='adagrad_lr', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=3))

\multirow{5}{*}{ADAGRAD}
& 0.1 & 89.190 ($\pm$ 0.160) & 0.447 ($\pm$ 0.017) \\ 
& 0.05 & 89.40 ($\pm$ 0.310) & 0.425 ($\pm$ 0.005) \\ 
& 0.01 & 88.770 ($\pm$ 0.090) & 0.440 ($\pm$ 0.014) \\ 
& 0.0075 & 88.395 ($\pm$ 0.025) & 0.440 ($\pm$ 0.011) \\ 
& 0.005 & 87.805 ($\pm$ 0.095) & 0.452 ($\pm$ 0.006) \\ 
\midrule 



In [86]:
# TAS experiments

tas_experiments = full_experiment.filter_parameters({'training_method': 'sgd',
                                                    'learning_method':'tas'})

# see all sgd experiments

print(tas_experiments.len)

# for experiment in tas_experiments.list:
#     print(experiment.parameters_dict['epochs])
#     print('\n')
    
# 63 tests, 9 for each epoch selected(100, 50, 25, 15, 10, 5. 100 repets twice. we will disconsidere the first batch)



63


In [89]:
tas_100 = tas_experiments.filter_parameters({'epochs':100})

experiment_list = [tas_100]
names_list = ['TAS']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter=['sgd_lr'], 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=2))


TypeError: unhashable type: 'list'

In [70]:
print(tas_experiments.list[9].learning_method_parameters)
tas_experiments.list[9].results_ds

{'tas_alpha': 10, 'tas_beta': 0.3, 'tas_gamma': 0.02}


<xarray.Dataset>
Dimensions:        (epoch: 100, execution: 2)
Coordinates:
  * epoch          (epoch) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * execution      (execution) int64 1 2
Data variables:
    train_acc1     (execution, epoch) float64 20.88 34.13 46.56 ... 99.88 99.87
    val_acc1       (execution, epoch) float64 27.54 39.53 50.22 ... 90.97 90.79
    train_loss     (execution, epoch) float64 2.196 1.73 1.447 ... 0.003 0.004
    val_loss       (execution, epoch) float64 1.907 1.654 1.377 ... 0.617 0.608
    learning_rate  (execution, epoch) float64 0.04768 0.04744 ... 0.001049

In [57]:
experiment_list = [tas_experiments]
names_list = ['TAS']


for name,experiment in zip(names_list, experiment_list):
    

    print(experiment.make_latex_table(parameter='tas_alpha', 
                                      result_list=['val_acc1', 'val_loss'],
                                      result_type_list=['max_mean', 'min_mean'],
                                      name=name,
                                      precision=3))

\multirow{18}{*}{TAS}
& 10 & 91.065 ($\pm$ 0.155) & 0.376 ($\pm$ 0.003) \\ 
& 10 & 91.445 ($\pm$ 0.145) & 0.386 ($\pm$ 0.003) \\ 
& 10 & 91.535 ($\pm$ 0.135) & 0.376 ($\pm$ 0.011) \\ 
& 25 & 91.205 ($\pm$ 0.455) & 0.366 ($\pm$ 0.004) \\ 
& 25 & 91.50 ($\pm$ 0.10) & 0.375 ($\pm$ 0.009) \\ 
& 25 & 91.705 ($\pm$ 0.175) & 0.374 ($\pm$ 0.001) \\ 
& 50 & 91.045 ($\pm$ 0.235) & 0.374 ($\pm$ 0.001) \\ 
& 50 & 91.465 ($\pm$ 0.025) & 0.372 ($\pm$ 0.018) \\ 
& 50 & 91.640 ($\pm$ 0.320) & 0.371 ($\pm$ 0.003) \\ 
& 10 & 91.080 ($\pm$ 0.010) & 0.382 ($\pm$ 0.008) \\ 
& 10 & 91.525 ($\pm$ 0.275) & 0.369 ($\pm$ 0.008) \\ 
& 10 & 91.740 ($\pm$ 0.240) & 0.381 ($\pm$ 0.007) \\ 
& 25 & 91.235 ($\pm$ 0.005) & 0.362 ($\pm$ 0.003) \\ 
& 25 & 91.735 ($\pm$ 0.125) & 0.362 ($\pm$ 0.011) \\ 
& 25 & 91.970 ($\pm$ 0.080) & 0.364 ($\pm$ 0.007) \\ 
& 50 & 91.470 ($\pm$ 0.020) & 0.345 ($\pm$ 0.004) \\ 
& 50 & 91.560 ($\pm$ 0.20) & 0.379 ($\pm$ 0.008) \\ 
& 50 & 91.745 ($\pm$ 0.035) & 0.380 ($\pm$ 0.011) \\ 
\midrule 

In [13]:
print(sgd_experiments.make_latex_table(parameter='sgd_lr', 
                                       result_list=['val_acc1', 'val_loss'],
                                       result_type_list=['max_mean'],
                                       name='SGD Momentum',
                                       precision=2))

\multirow{25}{*}{SGD Momentum}
& 2 & 10.04 ($\pm$ 0.00) \\ 
& 2 & 10.12 ($\pm$ 0.06) \\ 
& 1 & 10.06 ($\pm$ 0.03) \\ 
& 1 & 10.02 ($\pm$ 0.03) \\ 
& 0.5 & 10.02 ($\pm$ 0.04) \\ 
& 0.5 & 28.96 ($\pm$ 19.08) \\ 
& 0.25 & 86.06 ($\pm$ 0.09) \\ 
& 0.25 & 48.54 ($\pm$ 38.54) \\ 
& 0.05 & 90.85 ($\pm$ 0.29) \\ 
& 0.05 & 91.08 ($\pm$ 0.44) \\ 
& 0.01 & 90.52 ($\pm$ 0.27) \\ 
& 0.01 & 90.58 ($\pm$ 0.23) \\ 
& 0.001 & 86.29 ($\pm$ 0.23) \\ 
& 0.001 & 86.54 ($\pm$ 0.34) \\ 
& 2 & 48.94 ($\pm$ 38.92) \\ 
& 1 & 89.07 ($\pm$ 0.04) \\ 
& 0.5 & 89.64 ($\pm$ 0.32) \\ 
& 0.25 & 90.48 ($\pm$ 0.23) \\ 
& 0.05 & 90.00 ($\pm$ 0.12) \\ 
& 0.01 & 86.41 ($\pm$ 0.23) \\ 
& 0.001 & 79.12 ($\pm$ 0.21) \\ 
& 0.05 & 89.92 ($\pm$ 0.43) \\ 
& 0.05 & 90.31 ($\pm$ 0.13) \\ 
& 0.05 & 90.50 ($\pm$ 0.23) \\ 
& 0.05 & 90.84 ($\pm$ 0.13) \\ 
\midrule 



In [9]:
sgd_experiments.show_results('T&L', ['train_acc1', 'val_acc1'], ['max_mean'])


---------- EXPERIMENT 0 ----------
*** PARAMETERS ***
training method:  sgd
sgd_lr :  2
sgd_momentum :  0.9
sgd_weight_decay :  0
sgd_dampening :  0
sgd_nesterov :  True
learning method:  constant
*** RESULTS ***
train_acc1 10.326
val_acc1 10.04
---------- EXPERIMENT 1 ----------
*** PARAMETERS ***
training method:  sgd
sgd_lr :  2
sgd_momentum :  0.9
sgd_weight_decay :  0
sgd_dampening :  0
sgd_nesterov :  False
learning method:  constant
*** RESULTS ***
train_acc1 10.212
val_acc1 10.120000000000001
---------- EXPERIMENT 2 ----------
*** PARAMETERS ***
training method:  sgd
sgd_lr :  1
sgd_momentum :  0.9
sgd_weight_decay :  0
sgd_dampening :  0
sgd_nesterov :  True
learning method:  constant
*** RESULTS ***
train_acc1 10.258
val_acc1 10.059999999999999
---------- EXPERIMENT 3 ----------
*** PARAMETERS ***
training method:  sgd
sgd_lr :  1
sgd_momentum :  0.9
sgd_weight_decay :  0
sgd_dampening :  0
sgd_nesterov :  False
learning method:  constant
*** RESULTS ***
train_acc1 10.428
val

In [10]:
sgd_experiments.list

In [153]:
a = {'a':'1', 'b':'2'}
for key, value in a.items():
    a[key] = int(value)

In [154]:
a

{'a': 1, 'b': 2}

In [223]:
'{{{}}}'.format(1)

'{1}'